# Simulations for PMA abstract

## 0. Imports

In [1]:
import numpy as np
from oawidgets.plantgl import *
from openalea.plantgl.all import Material, Color3, Scene
import oawidgets.mtg

from openalea.archicrop.archicrop import ArchiCrop
from openalea.archicrop.display import build_scene, display_scene
import matplotlib.pyplot as plt
%gui qt

## 1. Set management parameters

Set parameters regarding the spatial (and temporal) configuration of the crop.

In [2]:
file_tec_xml = 'sorgho_tec.xml'
params_tec = ['densitesem', 'interrang']
tec_stics = read_xml_file(file_tec_xml, params_tec)
sowing_density = tec_stics['densitesem']
inter_row = 0.4

## 2. Format crop-scale growth and senescence dynamics

From crop-scale data either measured or simulated with a crop model, generate a dictionnary of dictionnaries for each day with the following values :
 - "Thermal time" (float): cumulated thermal time from beginning of simulation to current day (in °C.day)
 - "Phenology" (str): current phenological stage (germination, juvenile, exponential or repro)
 - "Plant leaf area" (float): plant leaf area (in cm²) 
 - "Leaf area increment" (float): leaf area increment compared to previous day (in cm²) 
 - "Plant senescent leaf area" (float): senescent plant leaf area (in cm²) 
 - "Senescent leaf area increment" (float): senescent leaf area increment compared to previous day (in cm²) 
 - "Plant height" (float): plant height (in cm) 
 - "Height increment" (float): height increment compared to previous day (in cm).

In [ ]:
stics_output_file = 'mod_ssorghum.sti'
daily_dynamics = read_sti_file(stics_output_file, sowing_density)

# Time series 
# for thermal time, plant leaf area, plant senescent leaf area and plant height
thermal_time = [value["Thermal time"] for value in daily_dynamics.values()]
leaf_area_plant = [value["Plant leaf area"] for value in daily_dynamics.values()]
sen_leaf_area_plant = [value["Plant senescent leaf area"] for value in daily_dynamics.values()]
height_canopy = [value["Plant height"] for value in daily_dynamics.values()]

In [ ]:
file_plt_xml = 'proto_sorghum_plt.xml'
params_sen = ['durvieF', 'ratiodurvieI']
sen_stics = read_xml_file(file_plt_xml, params_sen)
lifespan = sen_stics['durvieF'] # leaf lifespan from appearance in exponential phase
lifespan_early = sen_stics['ratiodurvieI'] * lifespan # leaf lifespan from appearance in juvenile phase

## 3. Set plant architectural parameters

Set topological, geometrical and developmental parameters, in a range corresponding a given species, found in literature.

In [6]:
archi = dict(
    nb_phy=12, # number of phytomers on the main stem
    
    # Stem
    height=max(height_canopy), # potential plant height
    stem_q=1.1, # parameter for ligule height geometric distribution along axis
    diam_base=2.5, # stem base diameter
    diam_top=1.5, # stem top diameter

    # Leaf area distribution along the stem  
    leaf_area=max(leaf_area_plant), # potential plant leaf area
    rmax=0.8, #relative position of largest leaf on the stem
    skew=0.0005, # skewness for leaf area distribution along axis

    # blade area
    wl=0.12, # leaf blade width-to-length ratio 
    klig=0.6, # parameter for leaf blade shape
    swmax=0.55, # relative position of maximal blade width
    f1=0.64, # parameter for leaf blade shape
    f2=0.92, # parameter for leaf blade shape

    # blade curvature
    insertion_angle=35, # leaf blade insertion angle
    scurv=0.7, #  relative position of inflexion point
    curvature=120, # leaf blade insertion-to-tip angle
    phyllotactic_angle=137.5, # phyllotactic angle
    phyllotactic_deviation=0, # half-deviation to phyllotactic angle

    # Development
    phyllochron=30, # phyllochron, i.e. stem element appearance rate
    plastochron=20, # plastochron, i.e. leaf blade appearance rate

    # Senescence 
    leaf_lifespan = [lifespan_early, lifespan], # leaf lifespan from appearance

    # Tillering
    nb_tillers=0, # number of tillers
    tiller_delay=1, # delay, as factor of phyllochron, between the appearance of a phytomer and the appearance of its tiller
    tiller_angle=30,
    reduction_factor=1 # reduction factor between tillers of consecutive order
)

## 4. Instantiate ArchiCrop object

In [7]:
plant = ArchiCrop(daily_dynamics=daily_dynamics, **archi)

## 5. Generate a potential plant 

Generate a potential plant from architectural parameters. 

This creates an MTG (Multi-scale Tree Graph, cf [doc](https://mtg.readthedocs.io/en/latest/user/tutorial.html)) as a property of the object plant.

## 6. Simulate growth and senescence of plants

Simulate growth and senescence of this plant following the daily dynamics.

In [11]:
growing_plant = plant.grow_plant()

In [ ]:
# Create a figure with two subplots side by side
fig, axes = plt.subplots(1, 2, figsize=(12, 6))  # 1 row, 2 columns

# Plot on the first subplot
axes[0].plot(time, [la-sen for la, sen in zip(LA_stics, sen_LA_stics)], color="black", label="LA STICS")
for result in fitting_sim['LA']:
    axes[0].plot(time, result)
axes[0].set_xlabel("Thermal time (°C.day)")
axes[0].set_ylabel("Leaf Area / plant")
axes[0].set_title("Leaf Area: 3D canopy vs. STICS")
axes[0].legend()

# Plot on the second subplot
axes[1].plot(time, height_stics, color="black", label="height STICS")
for result in fitting_sim['height']:
    axes[1].plot(time, result)
axes[1].set_xlabel("Thermal time (°C.day)")
axes[1].set_ylabel("Plant height")
axes[1].set_title("Plant height: 3D canopy vs. STICS")
axes[1].legend()

# Adjust layout
plt.tight_layout()

# Show the plot
plt.show()

## 7. Morphospace

In [12]:
times = [t for i,t in enumerate(thermal_time)]
mean_time = sum(times) / len(times)
positions_temp = [ (0, 1*(t-mean_time), 0) for t in times]
nice_green = Color3((50, 100, 0))
scene, _ = build_scene([g for i,g in enumerate(list(growing_plant.values()))], position=positions, senescence=True, leaf_material = Material(nice_green), stem_material=Material(nice_green))
PlantGL(scene)

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680, 65280, 255], background…